In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/DrugBank-RAG


/content/drive/MyDrive/DrugBank-RAG


In [3]:
!ls -R


.:
adv_nlp_project1.ipynb		 s1_EDA.ipynb
BERT_inference.py		 s2_chunking.py
chroma_db_data			 s3_create_vector_db.py
chunks				 s4_retrieval.py
config.py			 s5_chat.py
data				 s6_generate_rag_responses.py
drug_bank_bert_classifier_final  s7_evaluate_rag.py
main.py				 s8_summarize_metrics.py
prompts				 ui.py
__pycache__

./chroma_db_data:
371fed05-37cd-45a2-b9a7-19dd94b196d0  chroma.sqlite3

./chroma_db_data/371fed05-37cd-45a2-b9a7-19dd94b196d0:
data_level0.bin  header.bin  index_metadata.pickle  length.bin	link_lists.bin

./chunks:
drug_chunks_final.json

./data:
drugbank_reduced_dataset.csv  kaggle_drugbank_clean_dataset.csv
evaluation_dataset.jsonl      rag_vs_groundtruth.jsonl

./drug_bank_bert_classifier_final:
config.json	   special_tokens_map.json  tokenizer.json     vocab.txt
model.safetensors  tokenizer_config.json    training_args.bin

./prompts:
master_prompt.txt

./__pycache__:
chat_baseline.cpython-311.pyc  s4_retrieval.cpython-312.pyc
chat_baseline.cpython-312.pyc  s5_c

In [4]:
!pip install -q transformers accelerate sentence-transformers sentencepiece huggingface_hub bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 21.0 MB/s eta 0:00:00


In [5]:
!pip install -q faiss-cpu tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 12.8 MB/s eta 0:00:00


In [6]:
!pip install -q langchain langchain-huggingface langchain-chroma chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.8/473.8 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 12.6 MB/s et

In [7]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 102.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 136.2 MB/s eta 0:00:00


In [8]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # 0=all, 1=info, 2=warning, 3=error

!export TF_CPP_MIN_LOG_LEVEL=3

!export ABSL_MAX_LOG_SEVERITY=3

!export TRANSFORMERS_VERBOSITY=error

!env TF_CPP_MIN_LOG_LEVEL=3 ABSL_MAX_LOG_SEVERITY=3

SHELL=/bin/bash
NV_LIBCUBLAS_VERSION=12.5.3.2-1
NVIDIA_VISIBLE_DEVICES=all
COLAB_JUPYTER_TRANSPORT=ipc
NV_NVML_DEV_VERSION=12.5.82-1
NV_CUDNN_PACKAGE_NAME=libcudnn9-cuda-12
CGROUP_MEMORY_EVENTS=/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events
NV_LIBNCCL_DEV_PACKAGE=libnccl-dev=2.22.3-1+cuda12.5
NV_LIBNCCL_DEV_PACKAGE_VERSION=2.22.3-1
VM_GCE_METADATA_HOST=169.254.169.253
HOSTNAME=0084dd86054a
MODEL_PROXY_HOST=https://mp.kaggle.net
LANGUAGE=en_US
TBE_RUNTIME_ADDR=172.28.0.1:8011
COLAB_TPU_1VM=
GCE_METADATA_TIMEOUT=3
NVIDIA_REQUIRE_CUDA=cuda>=12.5 brand=unknown,driver>=470,driver<471 brand=grid,driver>=470,driver<471 brand=tesla,driver>=470,driver<471 brand=nvidia,driver>=470,driver<471 brand=quadro,driver>=470,driver<471 brand=quadrortx,driver>=470,driver<471 brand=nvidiartx,driver>=470,driver<471 brand=vapps,driver>=470,driver<471 brand=vpc,driver>=470,driver<471 brand=vcs,driver>=470,driver<471 brand=vws,driver>=470,driver<471 brand=cloudgaming,driver>=470,

In [9]:
import sys
sys.path.append("/content/drive/MyDrive/DrugBank-RAG")


In [10]:
import os
os.chdir("/content/drive/MyDrive/DrugBank-RAG")
os.listdir()


['s7_evaluate_rag.py',
 's6_generate_rag_responses.py',
 's1_EDA.ipynb',
 's2_chunking.py',
 's4_retrieval.py',
 's8_summarize_metrics.py',
 'main.py',
 '.DS_Store',
 'ui.py',
 's3_create_vector_db.py',
 '__pycache__',
 '.gradio',
 'chunks',
 'chroma_db_data',
 'data',
 'prompts',
 '.git',
 'adv_nlp_project1.ipynb',
 'drug_bank_bert_classifier_final',
 'config.py',
 'BERT_inference.py',
 's5_chat.py']

In [11]:
!pip install -q pyngrok

from pyngrok import ngrok

# 
ngrok.set_auth_token("TOKEN")


In [21]:
from pyngrok import ngrok

public_url = ngrok.connect(8000)
print(" NGROK PUBLIC URL:", public_url)


 NGROK PUBLIC URL: NgrokTunnel: "https://suggestibly-unstrewn-lashaun.ngrok-free.dev" -> "http://localhost:8000"


In [ ]:
!python main.py


E0000 00:00:1764166548.791314    9803 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764166548.802183    9803 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764166548.837248    9803 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764166548.837274    9803 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764166548.837278    9803 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764166548.837282    9803 computation_placer.cc:177] computation placer already registered. Please check linka